# 데이터 점검 → train/val 분할(txt 리스트) → 클래스 자동 추출 → data.yaml 생성

In [1]:
%xmode Plain


Exception reporting mode: Plain


In [1]:
pip install -U ultralytics


Note: you may need to restart the kernel to use updated packages.


In [3]:

import os, glob, random, yaml
from pathlib import Path

# ==== 너의 폴더 ====
IMG_DIR = r"E:\pytorch_env\YOLOv8x_dataset\images"
LBL_DIR = r"E:\pytorch_env\YOLOv8x_dataset\labels"

# ==== 분할 비율/시드 ====
VAL_RATIO = 0.1
SEED = 777
random.seed(SEED)

# ==== 유효 이미지 확장자 ====
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

# 1) 이미지-라벨 페어링
def stem(p): return Path(p).stem

images = [p for ext in IMG_EXTS for p in glob.glob(os.path.join(IMG_DIR, f"**/*{ext}"), recursive=True)]
images = sorted(set(images))
labels = glob.glob(os.path.join(LBL_DIR, "**/*.txt"), recursive=True)
label_map = {stem(p): p for p in labels}

pairs = []
miss_lbl, miss_img = [], []
for img in images:
    s = stem(img)
    if s in label_map:
        pairs.append((img, label_map[s]))
    else:
        miss_lbl.append(img)

# 라벨 파일이 있는데 이미지가 없는 경우도 체크
for s, p in label_map.items():
    # 이미지 경로 후보들(같은 stem, 다양한 확장자)
    found = any(os.path.exists(os.path.join(Path(IMG_DIR), *(Path(p).parts[len(Path(LBL_DIR).parts):-1]), s + e)) for e in IMG_EXTS)
    if not found:
        # 느슨하게 전체 이미지 폴더에서 stem 매칭
        if not any(stem(i)==s for i in images):
            miss_img.append(p)

print(f"총 이미지: {len(images)} | 총 라벨: {len(labels)} | 페어링 성공: {len(pairs)}")
if miss_lbl:
    print(f"[경고] 라벨 없는 이미지 {len(miss_lbl)}개 (학습에서 제외): 예) {miss_lbl[:3]}")
if miss_img:
    print(f"[경고] 이미지 없는 라벨 {len(miss_img)}개 (학습에서 제외): 예) {miss_img[:3]}")

assert len(pairs) > 0, "이미지-라벨 페어가 없습니다. 폴더 구성을 다시 확인하세요."

# 2) 클래스 자동 추출 (라벨 파일에서 가장 큰 클래스 id + 1)
def max_class_id_from_label(txt_path):
    max_id = -1
    with open(txt_path, "r", encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if not line: 
                continue
            # YOLO 포맷: cls cx cy w h
            parts = line.split()
            try:
                cid = int(float(parts[0]))
                if cid > max_id:
                    max_id = cid
            except:
                pass
    return max_id

global_max = -1
for _, lbl in pairs:
    m = max_class_id_from_label(lbl)
    if m > global_max:
        global_max = m

assert global_max >= 0, "라벨 파일에서 클래스 id를 찾지 못했습니다."
nc = global_max + 1
names = [f"class_{i}" for i in range(nc)]
print(f"추정 클래스 수 nc={nc}, names={names}")

# 3) train/val 분할(txt 리스트 작성)
random.shuffle(pairs)
val_count = max(1, int(len(pairs) * VAL_RATIO))
val_pairs = pairs[:val_count]
train_pairs = pairs[val_count:]

DATA_ROOT = str(Path(IMG_DIR).parent)  # E:\pytorch_env\YOLOv8x_dataset
LIST_DIR = os.path.join(DATA_ROOT, "_splits")
os.makedirs(LIST_DIR, exist_ok=True)

train_list = os.path.join(LIST_DIR, "train.txt")
val_list   = os.path.join(LIST_DIR, "val.txt")

with open(train_list, "w", encoding="utf-8") as ft:
    for img, _ in train_pairs:
        ft.write(img.replace("\\","/") + "\n")
with open(val_list, "w", encoding="utf-8") as fv:
    for img, _ in val_pairs:
        fv.write(img.replace("\\","/") + "\n")

print(f"train 이미지 수: {len(train_pairs)} | val 이미지 수: {len(val_pairs)}")
print(f"train 목록: {train_list}")
print(f"val   목록: {val_list}")

# 4) data.yaml 생성 (txt 리스트 경로를 그대로 사용)
DATA_YAML = os.path.join(DATA_ROOT, "data.yaml")
data_cfg = {
    "train": train_list.replace("\\","/"),
    "val": val_list.replace("\\","/"),
    # 필요시 test 리스트를 추가로 만들면 같은 방식으로 넣을 수 있음
    "nc": nc,
    "names": names,
}
with open(DATA_YAML, "w", encoding="utf-8") as f:
    yaml.safe_dump(data_cfg, f, allow_unicode=True, sort_keys=False)

print(f"생성된 data.yaml 경로: {DATA_YAML}")


ERROR! Session/line number was not unique in database. History logging moved to new session 3
총 이미지: 1472 | 총 라벨: 1472 | 페어링 성공: 1472
추정 클래스 수 nc=73, names=['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9', 'class_10', 'class_11', 'class_12', 'class_13', 'class_14', 'class_15', 'class_16', 'class_17', 'class_18', 'class_19', 'class_20', 'class_21', 'class_22', 'class_23', 'class_24', 'class_25', 'class_26', 'class_27', 'class_28', 'class_29', 'class_30', 'class_31', 'class_32', 'class_33', 'class_34', 'class_35', 'class_36', 'class_37', 'class_38', 'class_39', 'class_40', 'class_41', 'class_42', 'class_43', 'class_44', 'class_45', 'class_46', 'class_47', 'class_48', 'class_49', 'class_50', 'class_51', 'class_52', 'class_53', 'class_54', 'class_55', 'class_56', 'class_57', 'class_58', 'class_59', 'class_60', 'class_61', 'class_62', 'class_63', 'class_64', 'class_65', 'class_66', 'class_67', 'class_68', 'class_69', 'class_70', '

# YOLOv8-M 모델을 활용하여
객체 탐지(Object Detection) 학습을 2단계(저해상도 → 고해상도) 로 진행합니다.

Stage-A: 640px 해상도에서 기본 학습 (빠른 수렴, 기초 학습)

Stage-B: 960px 해상도에서 세밀한 파인튜닝 (정밀도 향상)

In [ ]:
from ultralytics import YOLO
import os, glob

# 경로 설정
DATA_YAML = r"E:\pytorch_env\YOLOv8x_dataset\data.yaml"
PROJECT   = r"E:\pytorch_env\ai05-level1-project\Exp"
EXP_A     = "pill_v8m_640"
EXP_B     = "pill_v8m_960_ft"

os.makedirs(PROJECT, exist_ok=True)

# Stage-A (base training at 640px)
model_a = YOLO("yolov8m.pt")

train_args_a = {
    "data": DATA_YAML,
    "imgsz": 640,
    "epochs": 80,
    "batch": 32,          
    "device": 0,
    "optimizer": "SGD",
    "lr0": 0.01, "lrf": 0.1,
    "warmup_epochs": 3,
    "cos_lr": True,
    "close_mosaic": 10,
    "hsv_h": 0.015, "hsv_s": 0.7, "hsv_v": 0.4,
    "degrees": 0.0, "translate": 0.08, "scale": 0.5, "shear": 0.0, "perspective": 0.0,
    "fliplr": 0.5, "flipud": 0.0,
    "mosaic": 0.2, "mixup": 0.0, "copy_paste": 0.0, "erasing": 0.0,
    "box": 7.0, "cls": 0.7, "dfl": 1.5,
    "weight_decay": 0.0005,
    "patience": 30,
    "cache": "ram",
    "workers": 4,
    "verbose": True,
    "project": PROJECT, "name": EXP_A,
}
results_a = model_a.train(**train_args_a)

# Stage-A best 모델 경로
model_a_best = getattr(model_a, "ckpt_path", None) or glob.glob(
    os.path.join(PROJECT, EXP_A, "weights", "best.pt")
)[0]
print(f" Best Stage-A model: {model_a_best}")

# Stage-B (fine-tune at 960px)
model_b = YOLO(model_a_best)

train_args_b = {
    "data": DATA_YAML,
    "imgsz": 960,
    "epochs": 40,
    "batch": 16,
    "device": 0,
    "optimizer": "SGD",
    "lr0": 0.003, "lrf": 0.05,
    "warmup_epochs": 2,
    "cos_lr": True,
    "close_mosaic": 100,
    "mosaic": 0.0, "mixup": 0.0, "scale": 0.2,
    "box": 7.5, "cls": 0.5, "dfl": 1.5,
    "weight_decay": 0.0005,
    "cache": "ram",
    "workers": 4,
    "verbose": True,
    "project": PROJECT, "name": EXP_B,
}
results_b = model_b.train(**train_args_b)

# Stage-B best 모델
model_b_best = getattr(model_b, "ckpt_path", None) or glob.glob(
    os.path.join(PROJECT, EXP_B, "weights", "best.pt")
)[0]
print(f" Best Stage-B model: {model_b_best}")

# 최종 검증
val_metrics = model_b.val(
    model=model_b_best,
    data=DATA_YAML,
    imgsz=960,
    conf=0.001,
    iou=0.70,
    plots=True,
    save_json=True,
)

try:
    print({k: float(v) for k, v in val_metrics.results_dict.items()})
except Exception as e:
    print("Validation metrics 출력 오류:", e)
    print(val_metrics)


Ultralytics 8.3.221  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.0, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.7, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=E:\pytorch_env\YOLOv8x_dataset\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.0, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=0.2, multi_scale=False, name=pill_v8m_640, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=30, perspective

# 예측

In [6]:
from ultralytics import YOLO

# 1️⃣ 학습 완료된 모델 로드
model_path = r"E:\pytorch_env\ai05-level1-project\Exp\pill_v8m_960_ft\weights\best.pt"
model = YOLO(model_path)

# 2️⃣ 테스트 이미지 폴더
test_dir = r"E:\pytorch_env\ai05-level1-project\test_images"

# 3️⃣ 예측 실행
results = model.predict(
    source=test_dir,
    imgsz=960,           # Stage-B와 동일한 크기
    conf=0.25,           # 감지 민감도 (필요 시 조정 가능)
    iou=0.5,
    save=True,
    save_txt=True,       # ✅ 반드시 켜야 csv 변환 가능
    project=r"E:\pytorch_env\ai05-level1-project\Predictions",
    name="pill_v8m_960_ft_test_pred"
)

print("✅ 예측 완료! 결과는 /Predictions/pill_v8m_test_pred 안에 저장되었습니다.")



image 1/843 E:\pytorch_env\ai05-level1-project\test_images\1.png: 960x736 1 class_0, 1 class_1, 1 class_7, 1 class_12, 64.8ms
image 2/843 E:\pytorch_env\ai05-level1-project\test_images\10.png: 960x736 1 class_0, 1 class_4, 1 class_9, 1 class_11, 6.2ms
image 3/843 E:\pytorch_env\ai05-level1-project\test_images\100.png: 960x736 1 class_0, 1 class_3, 1 class_4, 1 class_5, 6.5ms
image 4/843 E:\pytorch_env\ai05-level1-project\test_images\1003.png: 960x736 1 class_31, 1 class_50, 1 class_56, 1 class_61, 6.5ms
image 5/843 E:\pytorch_env\ai05-level1-project\test_images\1004.png: 960x736 1 class_31, 1 class_50, 1 class_56, 1 class_61, 7.6ms
image 6/843 E:\pytorch_env\ai05-level1-project\test_images\1005.png: 960x736 1 class_31, 1 class_50, 1 class_56, 1 class_61, 12.3ms
image 7/843 E:\pytorch_env\ai05-level1-project\test_images\1006.png: 960x736 1 class_34, 1 class_50, 1 class_58, 1 class_59, 10.6ms
image 8/843 E:\pytorch_env\ai05-level1-project\test_images\1007.png: 960x736 1 class_34, 1 clas

# 제출용 CSV로 변환

In [ ]:
from ultralytics import YOLO
import os
import pandas as pd

# ---------------------------------------------------
# 경로 설정
# ---------------------------------------------------
MODEL_PATH = r"E:\pytorch_env\ai05-level1-project\Exp\pill_v8m_640\weights\best.pt"
TEST_DIR   = r"E:\pytorch_env\ai05-level1-project\test_images"
OUTPUT_CSV = r"E:\pytorch_env\ai05-level1-project\submission\submission.csv"

# ---------------------------------------------------
# 클래스 매핑 (YOLO 내부 ID ↔ 실제 category_id)
# ※ 아래는 예시. 실제 category_id 매핑 테이블이 있다면 수정해야 함.
# ---------------------------------------------------
yolo_id_to_name = {i: str(i) for i in range(73)}  # YOLO class ID → name (기본)
name_to_original_id = {str(i): i for i in range(73)}  # name → 실제 category_id

# ---------------------------------------------------
# 모델 로드 및 예측 수행
# ---------------------------------------------------
model = YOLO(MODEL_PATH)
results = model.predict(
    source=TEST_DIR,
    imgsz=960,
    conf=0.001,
    iou=0.6,
    save=False,
    verbose=False
)

# ---------------------------------------------------
# 결과 파싱 및 submission.csv 생성
# ---------------------------------------------------
submission_data_yolo = []
ann_id = 1

for r in results:
    # 이미지 ID 추출 (파일명 숫자 기반)
    image_path = r.path
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    image_id = int(''.join(filter(str.isdigit, image_name)) or 0)

    boxes = r.boxes
    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes.xyxy[i].cpu().numpy()
        conf = boxes.conf[i].cpu().item()
        cls_id = int(boxes.cls[i].cpu().item())  # YOLO class ID (0~72)

        class_name = yolo_id_to_name.get(cls_id)
        if class_name is None:
            continue
        category_id = name_to_original_id.get(class_name)
        if category_id is None:
            continue

        submission_data_yolo.append({
            'annotation_id': ann_id,
            'image_id': image_id,
            'category_id': category_id,
            'bbox_x': int(round(x1)),
            'bbox_y': int(round(y1)),
            'bbox_w': int(round(x2 - x1)),
            'bbox_h': int(round(y2 - y1)),
            'score': round(conf, 4)
        })
        ann_id += 1

# ---------------------------------------------------
# CSV 저장
# ---------------------------------------------------
df = pd.DataFrame(submission_data_yolo, columns=[
    "annotation_id", "image_id", "category_id",
    "bbox_x", "bbox_y", "bbox_w", "bbox_h", "score"
])
os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)
df.to_csv(OUTPUT_CSV, index=False)

print(f"✅ 제출용 CSV 생성 완료: {OUTPUT_CSV}")
print(f"총 박스 수: {len(df)}")
print(df.head(10))


✅ 제출용 CSV 생성 완료: E:\pytorch_env\ai05-level1-project\submission\submission.csv
총 박스 수: 3936
   annotation_id  image_id  category_id  bbox_x  bbox_y  bbox_w  bbox_h  \
0              1         1            7     600     673     256     480   
1              2         1            0     158     252     203     125   
2              3         1            1     556      72     388     404   
3              4         1           12     172     740     181     293   
4              5        10           11     643     290     190     182   
5              6        10            4     101     807     242     239   
6              7        10            0     643     845     189     189   
7              8        10            9     103     237     408     219   
8              9       100            3     519     793     379     210   
9             10       100            4     587     131     230     226   

    score  
0  0.9685  
1  0.9652  
2  0.9562  
3  0.9475  
4  0.9764  
5  0.9734  